In [32]:
import pandas as pd
import Levenshtein

In [33]:
words = pd.read_csv('words.csv')

In [34]:
words.head()

,Français,Hiragana,Kanji
0,enseignant,きょうし,教室
1,étudiant,がくせい,学生
2,employé d'une compagnie,かいしゃいん,会社員
3,employé (avec nom de compagnie),しゃいん,社員
4,employé de banque,ぎんこういん,銀行員


In [36]:
shuffled_words = words.sample(frac=1).reset_index(drop=True)

In [78]:
def is_similar_japanese_answer(user_answer, correct_answer):
    similarity_threshold = 0.8
    if pd.isna(user_answer) or pd.isna(correct_answer):
        return False
    distance = Levenshtein.distance(user_answer.lower(), correct_answer.lower())
    similarity = 1 - (distance / max(len(user_answer), len(correct_answer)))
    return similarity >= similarity_threshold

In [79]:
def is_similar_french_answer(user_answer, correct_answer):
    similarity_threshold = 0.5
    if pd.isna(user_answer) or pd.isna(correct_answer):
        return False
    user_answer = user_answer.lower()
    correct_answer = correct_answer.lower()
    substring_match = user_answer in correct_answer or correct_answer in user_answer
    if substring_match:
        return True
    distance = Levenshtein.distance(user_answer, correct_answer)
    similarity = 1 - (distance / max(len(user_answer), len(correct_answer)))
    
    return similarity >= similarity_threshold

In [44]:
seen_fr=[]
correct_count = 0
shuffled_sample = shuffled_words.sample(n=10)
for i in range(len(shuffled_sample)):
    if shuffled_sample['Français'].iloc[i] not in seen_fr:
        ask_h = input(f"What's {shuffled_sample['Français'].iloc[i]}?")
        if is_similar_japanese_answer(ask_h, shuffled_sample['Hiragana'].iloc[i]):
            if shuffled_sample['Kanji'].iloc[i] != None:
                ask_k = input("And the kanji?")
                if is_similar_japanese_answer(ask_k, shuffled_sample['Kanji'].iloc[i]):
                    seen_fr.append(shuffled_sample['Français'].iloc[i])
                    correct_count += 1
                    print('Correct!')
                else:
                    print(f"Incorrect! The correct answer is {shuffled_sample['Kanji'].iloc[i]}")
                    break
        else:
            print(f"Incorrect! The correct answer is {shuffled_sample['Hiragana'].iloc[i]}")
            break

if correct_count == len(shuffled_sample):
    print("Congratulations! You answered all questions correctly.")

What's enseignant?きょうし
And the kanji?教室
Correct!
What's heureux de vous connaître?よろしくおねがいします
Incorrect! The correct answer is これからおせわになります


In [84]:
seen_k=[]
correct_count = 0
shuffled_sample = shuffled_words.sample(n=10)
for i in range(len(shuffled_sample)):
    kanji = shuffled_sample['Kanji'].iloc[i]
    if pd.notna(kanji) and kanji not in seen_k:
        ask_h = input(f"Qu'est-ce que ce kanji : {kanji}?")
        if is_similar_french_answer(ask_h, shuffled_sample['Français'].iloc[i]):
            seen_k.append(kanji)
            correct_count += 1
            print(f"Correct ! C'était bien {shuffled_sample['Français'].iloc[i]}")
        else:
            print(f"Incorrecte ! La bonne réponse était : {shuffled_sample['Français'].iloc[i]}")
            break

if correct_count == len(shuffled_sample):
    print("Félicitations ! Vous avez répondu correctement à toutes les réponses.")

Qu'est-ce que ce kanji : 喫茶店?salon de thé
Correct ! C'était bien café / salon de thé
Qu'est-ce que ce kanji : 上着?veste
Correct ! C'était bien veste
Qu'est-ce que ce kanji : 新聞?journal
Correct ! C'était bien journal
Qu'est-ce que ce kanji : 事務所?bureau
Correct ! C'était bien bureau
Qu'est-ce que ce kanji : 医者?médecin
Correct ! C'était bien médecin
Qu'est-ce que ce kanji : 下着?sous-vêtement
Correct ! C'était bien sous-vêtement
Qu'est-ce que ce kanji : 集めます?collectionner
Correct ! C'était bien collectionner / ramasser
Qu'est-ce que ce kanji : 急行?express
Correct ! C'était bien express
Qu'est-ce que ce kanji : 紅茶?thé anglais
Correct ! C'était bien thé anglais
Qu'est-ce que ce kanji : 郵便局?poste
Correct ! C'était bien bureau de poste
Félicitations ! Vous avez répondu correctement à toutes les réponses.
